# CNN Model

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.8 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist #inbuilt dataset in keras

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data() #it will download the images of train and test sets

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
#scaling down our images to 0 to 1
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
#reshaping the images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model=keras.Sequential([keras.layers.Conv2D(filters=hp.Int("conv_1_filter",min_value=32,max_value=128,step=16),kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),activation='relu',input_shape=(28,28,1)),
                         keras.layers.Conv2D(filters=hp.Int("conv_2_filter",min_value=32,max_value=128,step=16),kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),activation='relu'), #selecting the number of filters,kernel_size
                        keras.layers.Flatten(),
                        keras.layers.Dense(units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),activation='relu'), #selecting number of neurons
                        keras.layers.Dense(10,activation='softmax') 
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [9]:
from kerastuner import RandomSearch #this will find out which parameters will be best for our model
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [10]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='fashion_mnist')

In [11]:
tuner_search.search(train_images,train_labels,epochs=5,validation_split=0.1)

Trial 5 Complete [00h 03m 22s]
val_accuracy: 0.9116666913032532

Best val_accuracy So Far: 0.9153333306312561
Total elapsed time: 00h 11m 46s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        55344     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 32)                884768    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 941,722
Trainable params: 941,722
Non-trainable params: 0
_________________________________________________________________
